In [1]:
#Random Forest algorithm
#replicate findings from Walters et al 2014

In [3]:
#import io, os, sys, types
#from IPython import get_ipython
#from nbformat import read
#from IPython.core.interactiveshell import InteractiveShell

import biom
import sklearn
import pandas as pd

In [4]:
#HMP dataset
text_file = "/Users/silvalin/Desktop/merged_bmi_mapping_final__original_study_HMP__.txt"
biom_file = "/Users/silvalin/Desktop/filtered_otu_table__original_study_HMP__.biom"

In [5]:
m = pd.read_csv(text_file, sep="\t", index_col=0)

In [6]:
m.columns

Index(['BarcodeSequence', 'LinkerPrimerSequence', 'BMI', 'bmi_group_binned',
       'bmi_group_coded', 'original_study', 'combined_study_bmi_group',
       'PCR_PRIMERS', 'TARGET_SUBFRAGMENT', 'AGE', 'ELEVATION', 'LONGITUDE',
       'COUNTRY', 'SEQUENCING_METH', 'SAMPLE_CENTER', 'Description_duplicate',
       'ReversePrimer', 'COLLECTION_DATE', 'SEX', 'FAMILY_RELATIONSHIP_GG',
       'STUDY_CENTER', 'EXPERIMENT_CENTER', 'bmi_group_amish', 'RUN_CENTER',
       'LATITUDE', 'Description'],
      dtype='object')

In [7]:
m.index

Index(['SRS011271.396367', 'SRS011410.396107', 'SRS011415.394388',
       'SRS011529.396185', 'SRS011586.393765', 'SRS011621.394454',
       'SRS012191.395151', 'SRS013177.401537', 'SRS013386.400409',
       'SRS013543.394407',
       ...
       'SRS049157.403739', 'SRS049959.401022', 'SRS056620.394943',
       'SRS057447.396161', 'SRS063068.403097', 'SRS063524.397569',
       'SRS063827.396534', 'SRS065263.395822', 'SRS065466.395776',
       'SRS065725.395071'],
      dtype='object', name='#SampleID', length=208)

In [8]:
b= biom.load_table(biom_file)

In [9]:
#select only samples that fit LEAN OR OBESE
i = m[m.bmi_group_coded != "Overweight"].index
i_2 = m[m.bmi_group_coded != "Overweight"]

i_2.loc[i_2.bmi_group_binned == 3, 'bmi_group_binned'] = 0

/Users/silvalin/anaconda/lib/python3.5/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [10]:
# prints 4th column
y = m.iloc[:, 4]
print(y)
print(y.size)

#SampleID
SRS011271.396367        Normal
SRS011410.396107        Normal
SRS011415.394388        Normal
SRS011529.396185        Normal
SRS011586.393765        Normal
SRS011621.394454        Normal
SRS012191.395151        Normal
SRS013177.401537        Normal
SRS013386.400409        Normal
SRS013543.394407        Normal
SRS013687.400715        Normal
SRS013801.398248        Normal
SRS014345.393344        Normal
SRS014572.396411        Normal
SRS014885.394865        Normal
SRS014923.394014        Normal
SRS015247.394888        Normal
SRS015389.394987        Normal
SRS015599.394288        Normal
SRS015724.395613        Normal
SRS015782.394538        Normal
SRS015911.396089        Normal
SRS015960.395111        Normal
SRS016095.395879        Normal
SRS016267.395183        Normal
SRS016335.395358        Normal
SRS016400.399670        Normal
SRS016437.400001        Normal
SRS016517.399341        Normal
SRS016585.401158        Normal
                       ...    
SRS020926.402304    Overweigh

In [11]:
p = pd.DataFrame(b.matrix_data.T.todense().astype(int), index=b.ids(axis="sample"), columns=b.ids(axis="observation"))
print(p)

                  304309  193008  190299  191393  193061  186596  174924  \
SRS057447.396161       0       0       0       0       0       0       0   
SRS013762.393921       0       0       0       0       0       0       0   
SRS011415.394388       0       0       0       0       0       0       0   
SRS052697.393711       0       0       0       0       0       0       0   
SRS052433.395864       0       0       0       0       0       0       0   
SRS048722.395742       0       0       0       0       0       0       2   
SRS019910.396375       0       0       0       0       0       0       0   
SRS015247.394888       0       0       0       0       0       0       2   
SRS014999.393232       0       0       0       0       0       0       0   
SRS018968.393467       0       0       0       0       0       0       2   
SRS022480.394346       0       0       0       0       0       0       0   
SRS012191.395151       0       0       0       0       0       0       1   
SRS063921.39

In [12]:
c = p.loc[i, :]
print(c)

                  304309  193008  190299  191393  193061  186596  174924  \
#SampleID                                                                  
SRS011271.396367       0       0       0       0       0       0       0   
SRS011410.396107       0       0       0       0       0       0       2   
SRS011415.394388       0       0       0       0       0       0       0   
SRS011529.396185       0       0       0       0       0       0       0   
SRS011586.393765       0       0       0       0       0       0       0   
SRS011621.394454       0       0       0       0       0       0       0   
SRS012191.395151       0       0       0       0       0       0       1   
SRS013177.401537       0       0       0       0       0       0       0   
SRS013386.400409       0       0       0       0       0       0       0   
SRS013543.394407       0       0       0       0       0       0       1   
SRS013687.400715       0       0       0       0       0       0       0   
SRS013801.39

In [13]:
from sklearn import model_selection
from sklearn.ensemble import RandomForestClassifier
#10 fold cross-validation, 5 times repeated
#for i in range(5):
   # kfold = model_selection.KFold(n_splits=10, random_state=7)
    #model = RandomForestClassifier(n_estimators=1000)
    #results = model_selection.cross_val_score(model, p.fillna(0).as_matrix(), y, cv=kfold)
    #print(results.mean())

In [ ]:
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict
iter_range = range(1, 10)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=1000)
    #cross-validation score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_coded, cv=10, scoring='accuracy')
    predicted = cross_val_predict(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10)
    #cross-validation AUC score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    #rForest_scores.append(scores.mean())
    #average_accuracy+=scores.mean()
    print(confusion_matrix(i_2.bmi_group_binned, predicted))

#print(rForest_scores)
#print(average_accuracy/(5.0))

In [ ]:
from sklearn.cross_validation import cross_val_score
iter_range = range(1, 6)
# empty list to store scores
rForest_scores = []
average_accuracy = 0.0;

for i in iter_range:
    classifier = RandomForestClassifier(n_estimators=1000)
    #cross-validation score
    #scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_coded, cv=10, scoring='accuracy')
    #cross-validation AUC score
    scores = cross_val_score(classifier, c.fillna(0).as_matrix(), i_2.bmi_group_binned, cv=10, scoring='roc_auc')
    rForest_scores.append(scores.mean())
    average_accuracy+=scores.mean()

print(rForest_scores)
print(average_accuracy/(5.0))